In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def SPSA_ang(psi_est, NU_IT, fun, teo_val, s, t, a, A, b):
    """
    SPSA
    IN
        psi_est: estado inicial
        NU_IT: numero de iteraciones
        fun: funcion a optimizar
        teo_val: valor teorico
        s, t, a, A, b: parametros CSPSA
    OUT
        valor_optimo: valor optimo de la funcion en la iteracion k
        psi: estado resultante en la iteracion k
        error: valor absoluto entre valor teorioco y valor optimo en la iteracion k
    """

    n_par = int(psi_est.size/psi_est.shape[0])
    dim = int(psi_est.shape[0])
    
    valor_optimo = np.zeros((n_par, NU_IT))
    valor_optimo[:,0] = fun(psi_est)
    
    error = np.zeros((n_par, NU_IT))
    error[:,0] = abs(fun(psi_est)-teo_val)
    
    
    psi_optimo = np.zeros((NU_IT,dim,n_par), dtype='c16')
    psi_optimo[0,:,:] = psi_est

    for k in range(0,NU_IT-1):
        valor_optimo[:,k+1], error[:,k+1], psi_est = (itSPSA_ang(psi_est/np.linalg.norm(psi_est, axis = 0),
                                       k, s, t, a, A, b, fun, teo_val))
        psi_optimo[k+1,:,:] = psi_est
    return valor_optimo, error, psi_optimo

In [3]:
def itSPSA_ang(z_k,  k, s, t, a, A, b, fun, teo_val):
    """iteracion de SPSA
    IN
        z_k: dim x n_rep. Cada columna es un estimador de dimension dim.
        k: iteracion
        s, t, a, A, b: parametros SPSA
        fun: funcion a optimizar
    OUT
        z_k: dim x n_rep. Estimador actualizado
        z_value: real. valor de fun en el estimador z_k
    """
    
    alpha = a/(k+1+A)**s
    beta = b/(k+1)**t
    
    # Definir deltas para cada variable
    
    delta_t = (-1)**(np.random.randint(1,5))
    delta_p = (-1)**(np.random.randint(1,5))
    
    #obtener theta y phase
    
    z1 = z_k[0]
    r1 = np.absolute(z1)
    phi1 = np.angle(z1)

    z2 = z_k[1]
    r2 = np.absolute(z2)
    phi2 = np.angle(z2)

    R = np.divide(r2,r1)

    phase = float(phi2-phi1)
    theta = float(np.arctan(R))

    #sumamos y restamos delta
    
    theta_mas = theta + beta*delta_t
    theta_menos = theta - beta*delta_t
    
    phase_mas = phase + beta*delta_p
    phase_menos = phase - beta*delta_p
    
    # formamos estados normalizados para cada par theta,phase
    
    a1_mas = np.cos(theta_mas)
    a2_mas = np.sin(theta_mas)*(np.cos(phase_mas)+1j*np.sin(phase_mas))
    
    a1_menos = np.cos(theta_menos)
    a2_menos = np.sin(theta_menos)*(np.cos(phase_menos)+1j*np.sin(phase_menos))

    psi_bloch_mas = np.array([[a1_mas],[a2_mas]])
    psi_bloch_menos = np.array([[a1_menos],[a2_menos]])
    
    psi_bloch_mas = psi_bloch_mas/np.linalg.norm(psi_bloch_mas, axis=0)
    psi_bloch_menos = psi_bloch_menos/np.linalg.norm(psi_bloch_menos, axis=0)
    
    fun_k_mas = fun(psi_bloch_mas)
    fun_k_menos = fun(psi_bloch_menos)
    
   
    #Gradiente para ambas variables
    
    grad_t = np.divide(fun_k_mas - fun_k_menos, 2*beta*delta_t)
    grad_p = np.divide(fun_k_mas - fun_k_menos, 2*beta*delta_p)
    
    theta = theta - alpha*float(grad_t)
    phase = phase - alpha*float(grad_p)
    
    a1 = np.cos(theta)
    a2 = np.sin(theta)*(np.cos(phase)+1j*np.sin(phase))

    z_k = np.array([[a1],[a2]])
    z_k = z_k/np.linalg.norm(z_k, axis=0)
    
    z_value = fun(z_k)
    
    error = abs(z_value-teo_val)
    return z_value, error, z_k

In [4]:
def ganancia_a_SPSA_ang(z_k, l, fun, t, b):
    
    
    dim = z_k.shape[0]
    rep = int(z_k.size/z_k.shape[0])
    beta = b/(1)**t
    diff = np.empty((l,1,1), dtype=np.single)
    
    for i in range(0,l):
    
        # Definir deltas para cada variable
    
        delta_t = (-1)**(np.random.randint(1,5))
        delta_p = (-1)**(np.random.randint(1,5))
    
        #obtener theta y phase
    
        z1 = z_k[0]
        r1 = np.absolute(z1)
        phi1 = np.angle(z1)

        z2 = z_k[1]
        r2 = np.absolute(z2)
        phi2 = np.angle(z2)

        R = np.divide(r2,r1)

        phase = float(phi2-phi1)
        theta = float(np.arctan(R))

        #sumamos y restamos delta
    
        theta_mas = theta + beta*delta_t
        theta_menos = theta - beta*delta_t
    
        phase_mas = phase + beta*delta_p
        phase_menos = phase - beta*delta_p
    
        # formamos estados normalizados para cada par theta,phase
    
        a1_mas = np.cos(theta_mas)
        a2_mas = np.sin(theta_mas)*(np.cos(phase_mas)+1j*np.sin(phase_mas))
    
        a1_menos = np.cos(theta_menos)
        a2_menos = np.sin(theta_menos)*(np.cos(phase_menos)+1j*np.sin(phase_menos))

        psi_bloch_mas = np.array([[a1_mas],[a2_mas]])
        psi_bloch_menos = np.array([[a1_menos],[a2_menos]])
    
        psi_bloch_mas = psi_bloch_mas/np.linalg.norm(psi_bloch_mas, axis=0)
        psi_bloch_menos = psi_bloch_menos/np.linalg.norm(psi_bloch_menos, axis=0)
    
        fun_k_mas = fun(psi_bloch_mas)
        fun_k_menos = fun(psi_bloch_menos)
        
        diff[i] = abs(fun_k_mas-fun_k_menos)    
        
    prom = np.sum(diff, axis=0)/l
    
    a_arr = np.divide(2*np.pi*b, 5*prom)
    a_SPSA_ang = float(a_arr[0])
    
    return a_SPSA_ang

In [5]:
def opt_SPSA_ang(psi_est, NU_IT, fun, teo_val, s, t, A, b):
    """
    SPSA
    IN
        psi_est: dim x n_par matrix. dim is de dimension of each vector, and n_par
                the number of guesses in parallel.
        nu_ite: int. Number of iterations of SPSA.
        fun_teor: function. It is the known function, for example, the theoretical
                 fidelity between the unknown state of the system and the guess.
        fun: function. SPSA uses this function for the optimization.
        s, t, A, b: parametros SPSA
    OUT
        valor_optimo: real. Optimal value of fun finded by SPSA.
    """
    
    a = ganancia_a_SPSA_ang(psi_est, 25, fun, t, b)

    n_par = int(psi_est.size/psi_est.shape[0])
    dim = int(psi_est.shape[0])
    
    valor_optimo = np.zeros((n_par, NU_IT))
    valor_optimo[:,0] = fun(psi_est)
    
    error = np.zeros((n_par, NU_IT))
    error[:,0] = abs(fun(psi_est)-teo_val)
    
    
    psi_optimo = np.zeros((NU_IT,dim,n_par), dtype='c16')
    psi_optimo[0,:,:] = psi_est

    for k in range(0,NU_IT-1):
        valor_optimo[:,k+1], error[:,k+1], psi_est = (itSPSA_ang(psi_est/np.linalg.norm(psi_est, axis = 0),
                                       k, s, t, a, A, b, fun, teo_val))
        psi_optimo[k+1,:,:] = psi_est
    return valor_optimo, error, psi_optimo